# This cell is auto-generated.

In [ ]:
data = 0
while data not in [1, 2]:
    data = int(input('Which data set do you want to use? 1 = Test, 2 = Input '))

if data == 1:
    fp = 'Test/'
    fn = '2207test.txt'
elif data == 2:
    fp = 'Input/'
    fn = '2207input.txt'

f = open(fp + fn, 'r')
raw = [j for j in f.read().splitlines()]